# Day5_1: 이상치 및 데이터 정제 (Outliers & Data Cleaning)

## 📚 학습 목표

**Part 1: 기초 - 이상치 탐지**
1. IQR(사분위수) 방법으로 이상치 탐지하기
2. Z-score 방법으로 이상치 탐지하기
3. 박스플롯(px.box)으로 이상치 시각화하기
4. 산점도(px.scatter)로 이상치 확인하기
5. 이상치 제거 및 대체 전략 이해하기

**Part 2: 심화 - 데이터 정제**
1. astype()으로 데이터 타입 변환하기
2. pd.to_datetime()으로 날짜 데이터 변환하기
3. replace()로 잘못된 값 대체하기
4. Min-Max 정규화 및 Z-score 스케일링 적용하기

---

## 🎯 왜 이것을 배우나요?

| 개념 | 실무 활용 | 예시 |
|------|----------|------|
| 이상치 탐지 | 비정상 데이터 식별 | 주문액 1억원(오류), 나이 200세(오류) |
| 박스플롯 시각화 | 이상치 빠른 파악 | 월별 매출 분포에서 이상 거래 발견 |
| 데이터 타입 변환 | 연산 가능한 형태로 변경 | "1,000" → 1000 (문자열→숫자) |
| 정규화/스케일링 | 모델 학습 성능 향상 | 연봉(백만원), 나이(세) → 0~1 범위 통일 |

**분석가 관점**: 실무 데이터의 90%는 "더러운 데이터"입니다. 이상치 제거와 타입 변환 없이는 제대로 된 분석이 불가능합니다!

---

# Part 1: 기초 - 이상치 탐지

---

## 1.1 이상치란?

**이상치(Outlier)**는 전체 데이터 분포에서 크게 벗어난 값입니다.

- **입력 오류**: 나이 200세, 주문액 -5000원
- **측정 오류**: 센서 고장, 통신 에러
- **실제 극단값**: 억만장자의 연봉, 초대형 거래

**왜 중요한가?**
- 평균/표준편차를 왜곡시킴
- 머신러닝 모델 성능 저하
- 잘못된 비즈니스 의사결정

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

     월    매출
0   1월   120
1   2월   135
2   3월   128
3   4월  5000
4   5월   140
5   6월   132
6   7월   125
7   8월   138
8   9월   145
9  10월   130

평균: 619.3만원
중앙값: 133.5만원


In [2]:
# 샘플 데이터: 월별 매출 (단위: 만원)
sales_data = pd.DataFrame({
    '월': ['1월', '2월', '3월', '4월', '5월', '6월', '7월', '8월', '9월', '10월'],
    '매출': [120, 135, 128, 5000, 140, 132, 125, 138, 145, 130]  # 4월에 이상치!
})

print(sales_data)
print(f"\n평균: {sales_data['매출'].mean():.1f}만원")  # 이상치 때문에 왜곡됨
print(f"중앙값: {sales_data['매출'].median():.1f}만원")  # 이상치에 강함

     월    매출
0   1월   120
1   2월   135
2   3월   128
3   4월  5000
4   5월   140
5   6월   132
6   7월   125
7   8월   138
8   9월   145
9  10월   130

평균: 619.3만원
중앙값: 133.5만원


---

## 1.2 IQR(사분위수) 방법

**IQR (Interquartile Range)** = Q3 - Q1

- **Q1 (25 백분위수)**: 하위 25% 지점
- **Q3 (75 백분위수)**: 상위 25% 지점
- **이상치 범위**: `[Q1 - 1.5*IQR, Q3 + 1.5*IQR]` 밖의 값

가장 널리 쓰이는 이상치 탐지 방법입니다!

In [3]:
# IQR 방법으로 이상치 탐지
Q1 = sales_data['매출'].quantile(0.25)
Q3 = sales_data['매출'].quantile(0.75)
IQR = Q3 - Q1

# 이상치 범위 계산
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(f"Q1 (25%): {Q1}")
print(f"Q3 (75%): {Q3}")
print(f"IQR: {IQR}")
print(f"\n정상 범위: [{lower_bound:.1f}, {upper_bound:.1f}]")

# ⭐이상치⭐
outliers = sales_data[(sales_data['매출'] < lower_bound) | (sales_data['매출'] > upper_bound)]
print(f"\n이상치 발견:")
print(outliers)

Q1 (25%): 128.5
Q3 (75%): 139.5
IQR: 11.0

정상 범위: [112.0, 156.0]

이상치 발견:
    월    매출
3  4월  5000


### 💡 실무 예시: 전자상거래 주문액 이상치 탐지

쇼핑몰 데이터에서 비정상적으로 높은 주문액을 찾아 사기 거래를 방지합니다.

In [ ]:
#수업시간에 만들어 본 것

def search_outlier(df: pd.DataFrame, column: str) -> pd.DataFrame:
    q1= df[column].quantile(0.25)
    q3= df[column].quantile(0.75)
    IQR = q3 - q1
    lower_bound = q1 - 1.5*IQR
    upper_bound = q3 + 1.5*IQR
    outliers = df[
         (df[column] < lower_bound) #조건식1
        |
         (df[column] > upper_bound)  #조건식2
    ]
    return outliers

In [7]:
# 전자상거래 주문 데이터
orders = pd.DataFrame({
    '주문ID': ['O001', 'O002', 'O003', 'O004', 'O005', 'O006', 'O007', 'O008'],
    '주문액': [35000, 42000, 38000, 1500000, 40000, 37000, 45000, 39000]  # O004 의심!
})

Q1 = orders['주문액'].quantile(0.25)
Q3 = orders['주문액'].quantile(0.75)
IQR = Q3 - Q1
upper = Q3 + 1.5 * IQR

suspicious_orders = orders[orders['주문액'] > upper]
print("의심 거래:")
print(suspicious_orders)

의심 거래:
   주문ID      주문액
3  O004  1500000


---

## 1.3 Z-score 방법

**Z-score** = (값 - 평균) / 표준편차

- 평균으로부터 **몇 표준편차** 떨어져 있는지 측정
- 일반적으로 `|Z-score| > 3`이면 이상치로 간주
- **정규분포 가정**이 필요함

In [ ]:
# Z-score 계산
mean = sales_data['매출'].mean()
std = sales_data['매출'].std()

sales_data['Z-score'] = (sales_data['매출'] - mean) / std       #Z-score 공식

print(sales_data[['월', '매출', 'Z-score']])

# |Z-score| > 3인 이상치 찾기
outliers_z = sales_data[sales_data['Z-score'].abs() > 3]
print(f"\nZ-score 이상치:")
print(outliers_z[['월', '매출', 'Z-score']])

---

## 1.4 박스플롯으로 이상치 시각화 (px.box)

박스플롯은 **이상치를 한눈에 파악**할 수 있는 최고의 시각화 도구입니다.

- **박스**: Q1 ~ Q3 (IQR)
- **중간선**: 중앙값 (median)
- **수염**: 정상 범위
- **점**: 이상치

In [ ]:
# 박스플롯으로 이상치 시각화
fig = px.box(sales_data, y='매출', 
             title='월별 매출 박스플롯 (이상치 탐지)',
             labels={'매출': '매출 (만원)'})

fig.update_layout(
    height=500,
    showlegend=False
)

fig.show()

### 💡 실무 예시: 고객 연령대별 구매액 비교

연령대별 박스플롯으로 각 그룹의 이상치를 동시에 파악합니다.

In [ ]:
# 고객 구매 데이터
customer_purchases = pd.DataFrame({
    '연령대': ['20대']*10 + ['30대']*10 + ['40대']*10,
    '구매액': [
        30, 35, 32, 28, 40, 33, 37, 29, 500, 31,  # 20대 (500 이상치)
        50, 55, 52, 48, 60, 53, 57, 49, 54, 51,   # 30대
        70, 75, 72, 68, 80, 73, 77, 69, 74, 1000  # 40대 (1000 이상치)
    ]
})

fig = px.box(customer_purchases, x='연령대', y='구매액',
             title='연령대별 구매액 분포 (이상치 포함)',
             labels={'구매액': '구매액 (천원)'},
             color='연령대')

fig.show()

---

## 1.5 산점도로 이상치 확인 (px.scatter)

**2개 변수 간의 관계**에서 이상치를 찾을 때 산점도를 사용합니다.

- 예: 광고비 vs 매출, 나이 vs 연봉
- 추세선에서 크게 벗어난 점이 이상치

In [8]:
# 광고비 vs 매출 데이터
ad_data = pd.DataFrame({
    '광고비': [100, 150, 200, 250, 300, 350, 400, 450, 500, 180],
    '매출': [500, 750, 1000, 1250, 1500, 1750, 2000, 2250, 2500, 100]  # 마지막이 이상치
})

fig = px.scatter(ad_data, x='광고비', y='매출',
                 title='광고비 vs 매출 산점도 (이상치 탐지)',
                 labels={'광고비': '광고비 (만원)', '매출': '매출 (만원)'},
                 trendline='ols',  # 추세선 추가
                 size=[10]*9 + [30])  # 이상치 크기 강조

fig.update_traces(marker=dict(color=['blue']*9 + ['red']))  # 이상치 빨간색
fig.show()

### 💡 실무 예시: 아파트 면적 vs 가격 이상치 탐지

부동산 데이터에서 비정상적으로 싸거나 비싼 매물을 찾습니다.

In [9]:
# 아파트 데이터
apartments = pd.DataFrame({
    '면적': [60, 70, 80, 90, 100, 85, 75, 95, 65, 88],
    '가격': [300, 350, 400, 450, 500, 425, 375, 475, 50, 440],  # 9번째 비정상 저가
    '단지명': ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I(의심)', 'J']
})

fig = px.scatter(apartments, x='면적', y='가격', text='단지명',
                 title='아파트 면적 vs 가격 (이상치 탐지)',
                 labels={'면적': '면적 (㎡)', '가격': '가격 (백만원)'})

fig.update_traces(textposition='top center')
fig.show()

---

# Part 2: 심화 - 데이터 정제

---

## 2.1 데이터 타입 변환 (astype)

**문제**: 숫자처럼 보이지만 문자열로 저장된 데이터

- `"1000"` (문자열) → `1000` (정수)
- `"1,234"` → 쉼표 제거 후 변환
- `"True"` → `True` (불린)

In [ ]:
# 잘못된 타입의 데이터
dirty_data = pd.DataFrame({
    '제품ID': ['P001', 'P002', 'P003'],
    '가격': ['10000', '20000', '15000'],  # 문자열!
    '재고': ['100', '200', '150'],        # 문자열!
    '할인가능': ['True', 'False', 'True']  # 문자열!
})


print("원본 데이터 타입:")
print(dirty_data.dtypes)
print(f"\n가격 합계 시도: {dirty_data['가격'].sum()}")  # 문자열 연결됨!

In [ ]:
# 타입 변환
clean_data = dirty_data.copy()
clean_data['가격'] = clean_data['가격'].astype(int)
clean_data['재고'] = clean_data['재고'].astype(int)
clean_data['할인가능'] = clean_data['할인가능'].astype(bool)

print("변환된 데이터 타입:")
print(clean_data.dtypes)
print(f"\n가격 합계: {clean_data['가격'].sum():,}원")  # 정상 계산!

### 쉼표(,) 포함 숫자 변환

In [11]:
# 쉼표 포함 숫자
messy_numbers = pd.DataFrame({
    '제품': ['노트북', '마우스', '키보드'],
    '가격': ['1,500,000', '30,000', '80,000']
})

# 쉼표 제거 후 정수로 변환
messy_numbers['가격_정제'] = messy_numbers['가격'].str.replace(',', '').astype(int)
print(messy_numbers)
print(f"\n평균 가격: {messy_numbers['가격_정제'].mean():,.0f}원")

    제품         가격    가격_정제
0  노트북  1,500,000  1500000
1  마우스     30,000    30000
2  키보드     80,000    80000

평균 가격: 536,667원


---

## 2.2 날짜 데이터 변환 (pd.to_datetime)

**문제**: 날짜가 문자열로 저장되어 있으면 시계열 분석 불가

- `"2024-01-15"` (문자열) → `Timestamp` (날짜)
- 다양한 포맷 자동 인식
- 연/월/일 추출 가능

In [13]:
from dateutil import parser  ⭐이거 아주 유용한 모듈임⭐

parser.parse("2026-01/07")

datetime.datetime(2026, 1, 7, 0, 0)

In [ ]:
# 날짜 문자열 데이터
sales_log = pd.DataFrame({
    '주문일': ['2024-01-15', '2024-01-20', '2024-02-05', '2024-02-18'],
    '매출': [1000, 1200, 980, 1100]
})

print("원본 타입:")
print(sales_log['주문일'].dtype)  # object (문자열)

In [ ]:
# 날짜로 변환
sales_log['주문일'] = pd.to_datetime(sales_log['주문일'])

print("변환 후:")
print(sales_log['주문일'].dtype)  # datetime64[ns]

# 날짜 연산 가능!
sales_log['연도'] = sales_log['주문일'].dt.year
sales_log['월'] = sales_log['주문일'].dt.month
sales_log['요일'] = sales_log['주문일'].dt.day_name()

print("\n날짜 정보 추출:")
print(sales_log)

### 💡 실무 예시: 다양한 날짜 포맷 통일

In [ ]:
# 여러 포맷의 날짜
mixed_dates = pd.DataFrame({
    '입력일': ['2024-01-15', '01/20/2024', '2024.02.05', '2024년 02월 18일']
})

# Pandas가 자동으로 파싱 (대부분 성공)
mixed_dates['표준날짜'] = pd.to_datetime(mixed_dates['입력일'], errors='coerce')

# 한국어 포맷은 수동 처리 필요
mixed_dates.loc[3, '표준날짜'] = pd.to_datetime('2024-02-18')

print(mixed_dates)

---

## 2.3 값 대체 (replace)

**문제**: 잘못 입력된 값, 오타, 일관성 없는 표기

- `"남"`, `"M"`, `"male"` → `"남성"` 통일
- `999`, `-1` (결측 표시) → `NaN`
- 오타 수정

In [14]:
# 일관성 없는 데이터
messy_customers = pd.DataFrame({
    '이름': ['김철수', '이영희', '박민수'],
    '성별': ['남', 'F', 'M'],  # 표기 불일치!
    '등급': ['VIP', 'vip', 'VIP'],  # 대소문자 혼재
    '나이': [30, 999, 25]  # 999는 결측치 표시
})

print("원본 데이터:")
print(messy_customers)

원본 데이터:
    이름 성별   등급   나이
0  김철수  남  VIP   30
1  이영희  F  vip  999
2  박민수  M  VIP   25


In [ ]:
# 값 대체로 정제
clean_customers = messy_customers.copy()

# 성별 통일
clean_customers['성별'] = clean_customers['성별'].replace({
    '남': '남성',
    'M': '남성',        
    'F': '여성',
    '여': '여성'
})

# 등급 대문자 통일
clean_customers['등급'] = clean_customers['등급'].str.upper()

# 999를 NaN으로 변경
clean_customers['나이'] = clean_customers['나이'].replace(999, np.nan)

print("\n정제된 데이터:")
print(clean_customers)

---

## 2.4 정규화 및 스케일링

**왜 필요한가?**
- 변수마다 단위가 다르면 (연봉: 백만원, 나이: 세) 머신러닝 모델이 편향됨
- 모든 변수를 **동일한 스케일**로 맞춰야 공정한 비교 가능

### Min-Max 정규화
- 모든 값을 **0~1 범위**로 변환
- 공식: `(x - min) / (max - min)`⭐⭐⭐

In [ ]:
# 다양한 스케일의 데이터
employee_data = pd.DataFrame({
    '직원명': ['김철수', '이영희', '박민수', '최지영'],
    '연봉': [3000, 5000, 4000, 6000],  # 백만원 단위
    '나이': [28, 35, 30, 40],          # 세
    '경력': [2, 8, 5, 12]              # 년
})

print("원본 데이터:")
print(employee_data)

In [ ]:
# Min-Max 정규화 (0~1)
normalized = employee_data.copy()

for col in ['연봉', '나이', '경력']:
    min_val = employee_data[col].min()
    max_val = employee_data[col].max()
    normalized[f'{col}_정규화'] = (employee_data[col] - min_val) / (max_val - min_val)

print("\nMin-Max 정규화 (0~1 범위):")
print(normalized[['직원명', '연봉_정규화', '나이_정규화', '경력_정규화']])

### Z-score 스케일링 (표준화)
- 평균 0, 표준편차 1로 변환
- 공식: `(x - 평균) / 표준편차`
- **이상치에 민감**함

- 작아지긴 하지만 정규분포에 의해 퍼짐!
- 최소값과 최대값이 없음

In [17]:
# Z-score 표준화
standardized = employee_data.copy()

for col in ['연봉', '나이', '경력']:
    mean_val = employee_data[col].mean()
    std_val = employee_data[col].std()
    standardized[f'{col}_표준화'] = (employee_data[col] - mean_val) / std_val

print("\nZ-score 표준화 (평균 0, 표준편차 1):")
print(standardized[['직원명', '연봉_표준화', '나이_표준화', '경력_표준화']])

NameError: name 'employee_data' is not defined

### 💡 실무 예시: 전처리 전후 히스토그램 비교

정규화/표준화 효과를 시각화로 확인합니다.

In [16]:
# 원본 데이터 분포
fig = px.histogram(employee_data, x='연봉', nbins=10,
                   title='원본 연봉 분포 (단위: 백만원)',
                   labels={'연봉': '연봉 (백만원)'})
fig.show()

# 정규화 후 분포
fig2 = px.histogram(normalized, x='연봉_정규화', nbins=10,
                    title='정규화된 연봉 분포 (0~1 범위)',
                    labels={'연봉_정규화': '정규화 연봉'})
fig2.show()

NameError: name 'employee_data' is not defined

---

## 🎯 실습 퀴즈

**난이도**: ⭐ (쉬움) ~ ⭐⭐⭐⭐⭐ (어려움)

---

### Q1. IQR로 이상치 찾기 ⭐

**문제**: 다음 월별 방문자 수 데이터에서 IQR 방법으로 이상치를 찾으세요.

```python
visitors = pd.DataFrame({
    '월': ['1월', '2월', '3월', '4월', '5월', '6월'],
    '방문자수': [1000, 1050, 1020, 5000, 1080, 1040]
})
```

**기대 결과**: 이상치가 있는 행 출력 (4월)

In [ ]:
# 여기에 코드를 작성하세요


### Q2. Z-score 이상치 탐지 ⭐⭐

**문제**: 다음 주문액 데이터에서 Z-score 방법으로 이상치를 찾으세요 (|Z-score| > 2).

```python
orders = pd.DataFrame({
    '주문액': [30000, 35000, 32000, 150000, 33000, 31000, 34000]
})
```

In [ ]:
# 여기에 코드를 작성하세요


### Q3. 박스플롯으로 시각화 ⭐⭐

**문제**: 다음 데이터를 박스플롯으로 시각화하여 이상치를 파악하세요.

```python
data = pd.DataFrame({
    '제품': ['A', 'B', 'C', 'D', 'E', 'F'],
    '가격': [10000, 12000, 11000, 50000, 10500, 11500]
})
```

In [ ]:
# 여기에 코드를 작성하세요


### Q4. 문자열을 숫자로 변환 ⭐⭐

**문제**: 쉼표가 포함된 가격 문자열을 정수로 변환하고 평균을 구하세요.

```python
products = pd.DataFrame({
    '제품명': ['노트북', '마우스', '키보드'],
    '가격': ['1,200,000', '30,000', '80,000']
})
```

**기대 결과**: 평균 가격 출력

In [ ]:
# 여기에 코드를 작성하세요


### Q5. 날짜 변환 및 추출 ⭐⭐

**문제**: 주문일을 datetime으로 변환하고 '월' 정보를 추출하세요.

```python
sales = pd.DataFrame({
    '주문일': ['2024-01-15', '2024-02-20', '2024-03-10'],
    '매출': [1000, 1200, 980]
})
```

In [ ]:
# 여기에 코드를 작성하세요


---

## Part 2 심화 퀴즈

---

### Q6. 값 대체로 데이터 정제 ⭐⭐⭐

**문제**: 성별 표기를 통일하고, 999는 NaN으로 변경하세요.

```python
customers = pd.DataFrame({
    '이름': ['김철수', '이영희', '박민수'],
    '성별': ['남', 'F', 'M'],
    '나이': [30, 999, 25]
})
```

**변환 규칙**: 남/M → 남성, 여/F → 여성

In [ ]:
# 여기에 코드를 작성하세요


### Q7. Min-Max 정규화 ⭐⭐⭐

**문제**: 연봉과 나이를 0~1 범위로 정규화하세요.

```python
employees = pd.DataFrame({
    '직원명': ['김철수', '이영희', '박민수'],
    '연봉': [3000, 5000, 4000],
    '나이': [28, 35, 30]
})
```

**기대 결과**: 연봉_정규화, 나이_정규화 컬럼 추가

In [ ]:
# 여기에 코드를 작성하세요


### Q8. 산점도로 이상치 시각화 ⭐⭐⭐⭐

**문제**: 광고비와 매출 산점도를 그리고, 이상치를 빨간색으로 표시하세요.

```python
ad_sales = pd.DataFrame({
    '광고비': [100, 200, 300, 400, 500, 250],
    '매출': [500, 1000, 1500, 2000, 2500, 100]  # 마지막이 이상치
})
```

**힌트**: 마지막 점만 색상을 다르게

In [ ]:
# 여기에 코드를 작성하세요


### Q9. Z-score 표준화 ⭐⭐⭐⭐

**문제**: 매출 데이터를 Z-score로 표준화하고, 평균과 표준편차를 확인하세요.

```python
monthly_sales = pd.DataFrame({
    '매출': [1000, 1200, 1100, 1300, 1050, 1150]
})
```

**기대 결과**: 표준화 후 평균 ≈ 0, 표준편차 ≈ 1

In [ ]:
# 여기에 코드를 작성하세요


### Q10. 종합 데이터 정제 파이프라인 ⭐⭐⭐⭐⭐

**문제**: 다음 더러운 데이터를 완전히 정제하세요.

```python
dirty = pd.DataFrame({
    '주문일': ['2024-01-15', '2024-02-20', '2024-03-10'],
    '주문액': ['100,000', '200,000', '150,000'],
    '성별': ['M', 'F', 'M'],
    '나이': [999, 30, 25]
})
```

**정제 작업**:
1. 주문일을 datetime으로 변환하고 '월' 추출
2. 주문액에서 쉼표 제거하고 정수로 변환
3. 성별을 '남성', '여성'으로 통일
4. 나이 999를 NaN으로 변경
5. 주문액을 0~1로 정규화

**기대 결과**: 모든 컬럼이 정제된 DataFrame

In [ ]:
# 여기에 코드를 작성하세요


---

## 📊 학습 정리

### Part 1: 이상치 탐지 핵심 요약

| 방법 | 핵심 개념 | 실무 활용 |
|------|----------|----------|
| IQR | Q1 - 1.5*IQR ~ Q3 + 1.5*IQR | 가장 범용적, 분포 가정 불필요 |
| Z-score | \|Z\| > 3 (평균으로부터 3σ) | 정규분포 가정, 통계 분석용 |
| 박스플롯 | px.box() | 이상치 시각적 파악 |
| 산점도 | px.scatter() | 2변수 관계에서 이상치 탐지 |

### Part 2: 데이터 정제 핵심 요약

| 기법 | 문법 | 언제 쓰나? |
|------|------|----------|
| 타입 변환 | `astype(int)`, `str.replace()` | 문자열 숫자를 실제 숫자로 |
| 날짜 변환 | `pd.to_datetime()` | 시계열 분석, 날짜 연산 |
| 값 대체 | `replace({old: new})` | 표기 통일, 오타 수정 |
| Min-Max 정규화 | `(x - min) / (max - min)` | 0~1 범위, 딥러닝 전처리 |
| Z-score 표준화 | `(x - mean) / std` | 평균 0/표준편차 1, ML 전처리 |

### 💡 실무 팁

1. **이상치 처리 순서**: 탐지 → 시각화 → 원인 파악 → 제거/대체 결정
2. **IQR vs Z-score**: 분포 모르면 IQR, 정규분포면 Z-score
3. **타입 변환**: 항상 `errors='coerce'` 옵션으로 안전하게
4. **정규화 vs 표준화**: 딥러닝은 Min-Max, 전통 ML은 Z-score
5. **전후 비교**: 정제 전후 히스토그램/박스플롯 필수!